In [2]:
function power_method(A::AbstractMatrix, max_iter::Int=100, tol::Float64=1e-6)
    n = size(A, 1)
    x = rand(n)  # Start with a random initial vector
    x /= norm(x)  # Normalize
    λ_old = 0.0
    for iter in 1:max_iter
        x = A * x
        x /= norm(x)  # Normalize
        λ_new = dot(x, A * x) / dot(x, x)  # Rayleigh quotient
        if abs(λ_new - λ_old) < tol
            return true, x, λ_new  # Converged
        end
        λ_old = λ_new
    end
    return false, x, λ_old  # Did not converge within max_iter
end


power_method (generic function with 3 methods)

In [3]:
function qr_method(A::AbstractMatrix, max_iter::Int=100, tol::Float64=1e-6)
    Ak = A
    for iter in 1:max_iter
        Q, R = qr(Ak)  # QR decomposition
        Ak = R * Q  # Update Ak
        if maximum(abs.(Ak .- Diagonal(diag(Ak)))) < tol
            return true, diag(Ak)  # Converged: Return eigenvalues
        end
    end
    return false, diag(Ak)  # Did not converge within max_iter
end


qr_method (generic function with 3 methods)

In [4]:
using LinearAlgebra
using Random
using Printf

# Function to create symmetric matrices with specific properties
function generate_symmetric_matrix(size::Int, eigen_gap::Float64; random_state=42)
    Random.seed!(random_state)
    D = Diagonal([1.0, 1.0 / eigen_gap, fill(1.0 / (2 * eigen_gap), size - 2)...]) #Largest Eigenvalue is 1
    Q = qr(randn(size, size)).Q  # Random orthogonal matrix
    return Q * D * Q'  # Symmetric matrix
end

# Function to create a non-symmetric matrix
function generate_nonsymmetric_matrix(size::Int; random_state=42)
    Random.seed!(random_state)
    return randn(size, size)
end

# Function to test the Power and QR methods
function test_methods()
    sizes = [10, 100, 1000]  
    eigen_gaps = [25.0, 5.0, 1.0] 
    iterations = 100
    tolerance = 1e-6

    println("Testing Power and QR Methods\n")

    for size in sizes
        for eigen_gap in eigen_gaps
            # Generate a symmetric matrix
            A_sym = generate_symmetric_matrix(size, eigen_gap)
            println(@sprintf("Symmetric Matrix: size=%d, eigen_gap=%.2f", size, eigen_gap))

            # Test Power Method
            power_converged, power_vec, power_val = power_method(A_sym, iterations, tolerance)
            println("  Power Method converged? ", power_converged)
            println("  Largest eigenvalue (Power Method): ", power_val)

            # Test QR Method
            qr_converged, qr_vals = qr_method(A_sym, iterations, tolerance)
            println("  QR Method converged? ", qr_converged)
        end

        # Generate a non-symmetric matrix
        A_nonsym = generate_nonsymmetric_matrix(size)
        println(@sprintf("Non-symmetric Matrix: size=%d", size))

        # Test Power Method on non-symmetric matrix
        power_converged, _, _ = power_method(A_nonsym, iterations, tolerance)
        println("  Power Method converged? ", power_converged)

        # Test QR Method on non-symmetric matrix
        qr_converged, _ = qr_method(A_nonsym, iterations, tolerance)
        println("  QR Method converged? ", qr_converged)
    end
end

test_methods()

Testing Power and QR Methods

Symmetric Matrix: size=10, eigen_gap=25.00
  Power Method converged? true
  Largest eigenvalue (Power Method): 0.9999999999968708
  QR Method converged? true
Symmetric Matrix: size=10, eigen_gap=5.00
  Power Method converged? true
  Largest eigenvalue (Power Method): 0.9999999603381332
  QR Method converged? true
Symmetric Matrix: size=10, eigen_gap=1.00
  Power Method converged? true
  Largest eigenvalue (Power Method): 0.9999999069879087
  QR Method converged? true
Non-symmetric Matrix: size=10
  Power Method converged? false
  QR Method converged? false
Symmetric Matrix: size=100, eigen_gap=25.00
  Power Method converged? true
  Largest eigenvalue (Power Method): 0.9999999999957403
  QR Method converged? true
Symmetric Matrix: size=100, eigen_gap=5.00
  Power Method converged? true
  Largest eigenvalue (Power Method): 0.9999999998417708
  QR Method converged? true
Symmetric Matrix: size=100, eigen_gap=1.00
  Power Method converged? true
  Largest eigenv

In [5]:
function power_method_speed(A, max_iter=100, tol=1e-6)
    n = size(A, 1)
    x = rand(n)
    x /= norm(x)
    λ_old = 0.0
    iter_count = 0

    for k in 1:max_iter
        x = A * x
        x /= norm(x)
        λ_new = dot(x, A * x) / dot(x, x)
        iter_count += 1
        if abs(λ_new - λ_old) < tol
            return iter_count, x, λ_new
        end
        λ_old = λ_new
    end
    return max_iter, x, λ_old
end


power_method_speed (generic function with 3 methods)

In [6]:
function qr_method_speed(A, max_iter=100, tol=1e-6)
    Ak = A
    n = size(A, 1)
    iter_count = 0

    for k in 1:max_iter
        Q, R = qr(Ak)
        Ak = R * Q
        iter_count += 1
        if maximum(abs.(Ak .- Diagonal(diag(Ak)))) < tol
            return iter_count, diag(Ak)
        end
    end
    return max_iter, diag(Ak)
end


qr_method_speed (generic function with 3 methods)

In [8]:
using LinearAlgebra
using Random
using Printf

# Function to create symmetric matrices with specific properties
function generate_symmetric_matrix(size::Int, eigen_gap::Float64; random_state=42)
    Random.seed!(random_state)
    D = Diagonal([1.0, 1.0 / eigen_gap, fill(1.0 / (2 * eigen_gap), size - 2)...]) #Largest Eigenvalue is 1
    Q = qr(randn(size, size)).Q  # Random orthogonal matrix
    return Q * D * Q'  # Symmetric matrix
end

# Function to test Power Method Speed and QR Method Speed
function test_speed_methods()
    sizes = [10, 100, 1000] 
    eigen_gaps = [10.0, 5.0, 2.0, 1.0]  
    iterations = 100
    tolerance = 1e-6

    println("Testing Power Method Speed and QR Method Speed\n")

    for size in sizes
        for eigen_gap in eigen_gaps
            # Generate a symmetric matrix
            A = generate_symmetric_matrix(size, eigen_gap)
            println(@sprintf("Symmetric Matrix: size=%d, eigen_gap=%.2f", size, eigen_gap))

            # Test Power Method Speed
            power_iter_count, power_vec, power_val = power_method_speed(A, iterations, tolerance)
            println(@sprintf("  Power Method: iterations=%d, eigenvalue=%.6f", power_iter_count, power_val))

            # Test QR Method Speed
            qr_iter_count, qr_vals = qr_method_speed(A, iterations, tolerance)
            println(@sprintf("  QR Method: iterations=%d, eigenvalues=%.6f, ...", qr_iter_count, qr_vals[1]))
        end
    end
end

# Call the test function
test_speed_methods()


Testing Power Method Speed and QR Method Speed

Symmetric Matrix: size=10, eigen_gap=10.00
  Power Method: iterations=4, eigenvalue=1.000000
  QR Method: iterations=20, eigenvalues=1.000000, ...
Symmetric Matrix: size=10, eigen_gap=5.00
  Power Method: iterations=5, eigenvalue=1.000000
  QR Method: iterations=21, eigenvalues=1.000000, ...
Symmetric Matrix: size=10, eigen_gap=2.00
  Power Method: iterations=10, eigenvalue=1.000000
  QR Method: iterations=22, eigenvalues=1.000000, ...
Symmetric Matrix: size=10, eigen_gap=1.00
  Power Method: iterations=12, eigenvalue=1.000000
  QR Method: iterations=25, eigenvalues=1.000000, ...
Symmetric Matrix: size=100, eigen_gap=10.00
  Power Method: iterations=5, eigenvalue=1.000000
  QR Method: iterations=22, eigenvalues=1.000000, ...
Symmetric Matrix: size=100, eigen_gap=5.00
  Power Method: iterations=6, eigenvalue=1.000000
  QR Method: iterations=23, eigenvalues=1.000000, ...
Symmetric Matrix: size=100, eigen_gap=2.00
  Power Method: iterations=